# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [13]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

data = Dataset.load_from_df(new_df, Reader())

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [14]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [15]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [22]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

param_grid = {'n_factors' : [25, 100],
              'n_epochs' : [10, 20],
              'init_mean' : [0, 5, 10]}

gs = GridSearchCV(SVD, param_grid, n_jobs=-1)
gs.fit(data)

In [25]:
print(gs.best_score)
print(gs.best_params)

{'rmse': 0.8688918398442766, 'mae': 0.6677875288959878}
{'rmse': {'n_factors': 25, 'n_epochs': 20, 'init_mean': 0}, 'mae': {'n_factors': 25, 'n_epochs': 20, 'init_mean': 0}}


In [12]:
# print out optimal parameters for SVD after GridSearch

{'rmse': 0.8689250510051669, 'mae': 0.6679404366294037}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 100, 'reg_all': 0.05}}


In [29]:
# cross validating with KNNBasic
knn_basic = KNNBasic(name='pearson', user_based=True)
cv = cross_validate(knn_basic, data, n_jobs=-1)

In [30]:
cv

{'test_rmse': array([0.9455378 , 0.9480233 , 0.95177358, 0.94192277, 0.95197822]),
 'test_mae': array([0.72514082, 0.72706967, 0.72951475, 0.72427616, 0.72914054]),
 'fit_time': (0.060184478759765625,
  0.05990099906921387,
  0.06087684631347656,
  0.0571904182434082,
  0.05771517753601074),
 'test_time': (1.1562838554382324,
  1.1533751487731934,
  1.1506588459014893,
  1.1478197574615479,
  1.1436731815338135)}

In [32]:
import numpy as np

In [34]:
# print out the average RMSE score for the test set
print(cv)
print('-'*20)
print(np.mean(cv['test_rmse']))

{'test_rmse': array([0.9455378 , 0.9480233 , 0.95177358, 0.94192277, 0.95197822]), 'test_mae': array([0.72514082, 0.72706967, 0.72951475, 0.72427616, 0.72914054]), 'fit_time': (0.060184478759765625, 0.05990099906921387, 0.06087684631347656, 0.0571904182434082, 0.05771517753601074), 'test_time': (1.1562838554382324, 1.1533751487731934, 1.1506588459014893, 1.1478197574615479, 1.1436731815338135)}
--------------------
0.9478471335823073


In [36]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(name='pearson', user_based = True)
cv_knn_baseline = cross_validate(knn_baseline, data)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [37]:
# print out the average score for the test set
np.mean(cv_knn_baseline['test_rmse'])

0.8748726029829538

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [38]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [39]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [41]:
svd = SVD(n_factors= 25, n_epochs=20)
svd.fit(dataset)

In [42]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.06443253773952, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [86]:
def movie_rater(movie_df,num, genre=None):
    ratings = []
    rate_df = movie_df.sample(num)
    for i in range(num):
        print(rate_df[i:i+1])
        print('How do you rate this movie on a scale of 1-5, press n if you have not seen it :')
        rating = input()
        
        if (rating != 'n') and (int(rating) >= 1) and (int(rating) <=5):
            ratings.append({'userId':1000,
                            'movieId':rate_df.iloc[i].movieId,
                            'rating':rating})
    return ratings
        

In [87]:
ratings = movie_rater(df_movies, 4)
ratings

      movieId                     title         genres
3254     4397  Cannonball Run II (1984)  Action|Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen it :
2
      movieId                                  title                 genres
1483     2008  This World, Then the Fireworks (1997)  Crime|Drama|Film-Noir
How do you rate this movie on a scale of 1-5, press n if you have not seen it :
3
      movieId                 title          genres
1309     1755  Shooting Fish (1997)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen it :
4
      movieId                title                genres
5413    25805  Atalante, L' (1934)  Comedy|Drama|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen it :
5


[{'userId': 1000, 'movieId': 4397, 'rating': '2'},
 {'userId': 1000, 'movieId': 2008, 'rating': '3'},
 {'userId': 1000, 'movieId': 1755, 'rating': '4'},
 {'userId': 1000, 'movieId': 25805, 'rating': '5'}]

In [37]:
# try out the new function here!

      movieId                   title          genres
6579    55245  Good Luck Chuck (2007)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
5
      movieId                       title          genres
1873     2491  Simply Irresistible (1999)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId                  title  genres
3459     4718  American Pie 2 (2001)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
      movieId             title                   genres
4160     5990  Pinocchio (2002)  Children|Comedy|Fantasy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [38]:
user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [95]:
## add the new ratings to the original ratings DataFrame
append_df = new_df.append(ratings)
append_df

,userId,movieId,rating
0,1,1,4
1,1,3,4
2,1,6,4
3,1,47,5
4,1,50,5
...,...,...,...
100835,610,170875,3
0,1000,4397,2
1,1000,2008,3
2,1000,1755,4


In [99]:
new_data = Dataset.load_from_df(new_df, Reader())
new_train_data = new_data.build_full_trainset()

In [100]:
# train a model using the new combined DataFrame
svd2 = SVD(n_factors=25, n_epochs=20)
svd2.fit(new_train_data)

In [105]:
svd2.predict(1000, 1850)[3]

3.501556983616962

In [107]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
movies = []
for idx in new_df['movieId'].unique():
    movies.append( (idx, svd2.predict(1000, idx)[3]) )

In [108]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(movies, key=lambda x: x[1], reverse=True)
ranked_movies[:10]

[(318, 4.4194671252924),
 (750, 4.392835020695772),
 (1204, 4.382617410523673),
 (904, 4.333637401364741),
 (1104, 4.318658852543607),
 (2959, 4.304805265364703),
 (858, 4.295955545720456),
 (898, 4.295695013400326),
 (50, 4.283134680248461),
 (1213, 4.278983876246471)]

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [109]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [115]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for i in range(n):
        movie_id, rating = user_ratings[i]
        movie_title_df.set_index('movieId')
        movie_name = movie_title_df.loc[movie_id].title
        print(f'Recomendation # {n+1} : {movie_id} {movie_name}, {rating}')
            
recommended_movies(ranked_movies,df_movies,5)

Recomendation # 6 : 318 I Love Trouble (1994), 4.4194671252924
Recomendation # 6 : 750 Pompatus of Love, The (1996), 4.392835020695772
Recomendation # 6 : 1204 Leave It to Beaver (1997), 4.382617410523673
Recomendation # 6 : 904 Withnail & I (1987), 4.333637401364741
Recomendation # 6 : 1104 Cement Garden, The (1993), 4.318658852543607


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.